# IaaS Web Logs Review Template - Sentinel
BTV Project Obsidian, 2023

Author: juju43, https://blueteamvillage.org/programs/project-obsidian/ https://discord.gg/blueteamvillage
<img align="right" width="100" height="100" src="https://cfc.blueteamvillage.org/media/call-for-content-2021/img/20200622_BTVillage_logos_RGB_pos_hcOC7Qx.png">

This playbook is to help validating available logs
It helps to baseline environment, identify gaps and control points.

It targets Defcon31 BlueTeamVillage Project obsidian environment and splunk platform but it can be adapt to other logging platforms.

Possible sources
* IIS (W3CIISLog table). Like for most logs, it is critical that logs configuration is appropriate. Over time, Microsoft added user-agent and referer in default logs. If your webserver is behind a load balancer, web application firewall, or similar, you should set and log X-Forwarded-For to have real source IP of the request.

## Resources

* https://github.com/microsoft/msticpy/
* https://infosecjupyterthon.com/
* https://dropbox.tech/security/how-dropbox-security-builds-better-tools-for-threat-detection-and-incident-response
* [Log Files for a Web Site &lt;logFile&gt; - IIS](https://learn.microsoft.com/en-us/iis/configuration/system.applicationhost/sites/site/logfile/)
* [Adding Custom Fields to a Log File for a Site &lt;add&gt; - IIS](https://learn.microsoft.com/en-us/iis/configuration/system.applicationhost/sites/site/logfile/customfields/add)
* Webshell detection
  * https://github.com/nsacyber/Mitigating-Web-Shells
  * https://www.microsoft.com/en-us/security/blog/2020/02/04/ghost-in-the-shell-investigating-web-shell-attacks/
  * https://www.microsoft.com/en-us/security/blog/2021/02/11/web-shell-attacks-continue-to-rise/
* cli usage
  * as is `jupyter run notebook.ipynb --allow-errors` - https://docs.jupyter.org/en/latest/running.html#using-a-command-line-interface
  * with parameters `papermill input.ipynb output.ipynb -p alpha 0.6 -p l1_ratio 0.1` - https://papermill.readthedocs.io/en/latest/usage-workflow.html 

## Findings

_Put your findings here_

## Table of Contents

* Import
* Configuration
* Queries


## Import

In [ ]:
# Check we are running Python 3.6
import sys
MIN_REQ_PYTHON = (3,6)
if sys.version_info < MIN_REQ_PYTHON:
    print('Check the Kernel->Change Kernel menu and ensure that Python 3.6')
    print('or later is selected as the active kernel.')
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)

In [ ]:
# Imports
import pandas as pd
import msticpy.nbtools as nbtools
from datetime import datetime,timedelta
import os

In [ ]:
# path to config file
os.environ['MSTICPYCONFIG'] = '/home/ubuntu/msticpyconfig.yaml'
from msticpy.nbtools import *
from msticpy.data.data_providers import QueryProvider
from msticpy.common.wsconfig import WorkspaceConfig
from msticpy.nbtools.data_viewer import DataViewer
from msticpy.vis.matrix_plot import plot_matrix
from msticpy.nbtools import process_tree as ptree
from msticpy.context.geoip import GeoLiteLookup, IPStackLookup
print('Imports Complete')

## Configuration

In [ ]:
# Interactive settings edit
# https://msticpy.readthedocs.io/en/latest/getting_started/SettingsEditor.html#using-mpconfigfile-to-check-and-manage-your-msticpyconfig-yaml
from msticpy.config import MpConfigFile, MpConfigEdit, MpConfigControls
mpconfig = MpConfigFile()
# mpconfig.load_default()
# mpconfig.view_settings()
mpconfig

In [ ]:
# q_times = nbwidgets.QueryTime(units='hours', max_before=72, before=1, max_after=0)
q_times = nbwidgets.QueryTime(origin_time=datetime(2023, 4, 29), units='days', max_before=30, before=1, max_after=0)
#q_times = nbwidgets.QueryTime(origin_time=datetime(2023, 6, 15), units='days', max_before=1, before=0, max_after=0)
# q_times = nbwidgets.QueryTime(origin_time=datetime(2023, 6, 15), units='hours', max_before=4, before=0, max_after=0)

q_times.display()

In [ ]:
# If your environment footpring is very large or timeperiod too big, queries not optimized enough may return 'ADX query timed out' or 'Unknown query error' when done through msticpy.
# Ensure to use appropriate filters
query_common_args = ''
# query_common_args = f'''| where _SubscriptionId in ("12345", "67890")'''
results_limit = 10
vuln_scanners_ip = []

In [ ]:
query_common_args = query_common_args.strip()
query_common_args = query_common_args + f'''| where TimeGenerated >= datetime({q_times.start})
| where TimeGenerated <= datetime({q_times.end})'''
if vuln_scanners_ip:
    vuln_scanner_exclude_and = f'''and not (cIP in ("{'","'.join(vuln_scanners_ip)}"))'''
else:
    vuln_scanners_ip = ''

In [ ]:
# Configuration
qry_prov = QueryProvider("AzureSentinel")

In [ ]:
# Get the default Microsoft Sentinel workspace details from msticpyconfig.yaml
ws_config = WorkspaceConfig()

# Connect to Microsoft Sentinel with our QueryProvider and config details
qry_prov.connect(ws_config)

In [ ]:
# pandas
pd.set_option('display.max_colwidth', 500)

## Queries

### Timeperiod

Let's confirm that we have logs for the targeted timeperiod.

In [ ]:
q_times.start

In [ ]:
q_times.end

In [ ]:
query = f'''W3CIISLog {query_common_args}
| summarize max(TimeGenerated),min(TimeGenerated)
'''
df_timeperiod = qry_prov.exec_query(query)
df_timeperiod.head(results_limit)

### Trend

In [ ]:
query = f'''W3CIISLog
| where TimeGenerated >= ago(30d)
| summarize RequestCount = count() by bin(TimeGenerated, 1h)
| project TimeGenerated, RequestCount
'''
df_trend = qry_prov.exec_query(query)
df_trend.head(results_limit)

In [ ]:
# https://msticpy.readthedocs.io/en/latest/visualization/TimeSeriesAnomalies.html
from msticpy.analysis import timeseries

df_trend = df_trend.set_index("TimeGenerated")
ts_decomp_df = df_trend.mp_timeseries.analyze(
    # time_column="TimeGenerated"  - if the DF is not indexed by timestamp
    data_column="RequestCount",
    seasonal=7,
    period=24
)

ts_decomp_df.head()

In [ ]:
ts_decomp_df.mp_timeseries.plot(
    y="RequestCount",
)

### Misc

In [ ]:
query = f'''W3CIISLog
{query_common_args}
| summarize count() by sSiteName
| sort by count_ desc
| limit {results_limit}'''
df_iis_sites = qry_prov.exec_query(query)
df_iis_sites.head(results_limit)

In [ ]:
query = f'''W3CIISLog {query_common_args}
| summarize count() by sIP
| sort by count_ desc
| limit {results_limit}'''
df_iis_sIP = qry_prov.exec_query(query)
df_iis_sIP.head(results_limit)

In [ ]:
query = f'''W3CIISLog {query_common_args}
| summarize count() by csMethod
| sort by count_ desc
| limit {results_limit}'''
df_iis_csMethod = qry_prov.exec_query(query)
df_iis_csMethod.head(results_limit)

In [ ]:
query = f'''W3CIISLog {query_common_args}
| summarize count() by scStatus
| sort by count_ desc
| limit {results_limit}'''
df_iis_scStatus = qry_prov.exec_query(query)
df_iis_scStatus.head(results_limit)

In [ ]:
query = f'''W3CIISLog {query_common_args}
| summarize count() by csUriStem
| sort by count_ desc
'''
df_iis_csUriStem = qry_prov.exec_query(query)
df_iis_csUriStem.head(results_limit)

In [ ]:
df_iis_csUriStem.head(20)

In [ ]:
df_iis_csUriStem.tail(20)

In [ ]:
# Remove some noise? GUID, others.
query = r'''let regex = '[({]?[a-fA-F0-9]{8}[-]?([a-fA-F0-9]{4}[-]?){3}[a-fA-F0-9]{12}[})]?';
''' + f'''W3CIISLog {query_common_args}''' + r'''
| extend csUriQuery2 = replace(regex, 'GUID', csUriQuery)
| extend csUriQuery3 = replace('r=[0-9]{5,10}', 'r=NNN', csUriQuery2)
| extend csUriQuery4 = replace('_=[0-9]{9,13}', '_=NNN', csUriQuery3)
| extend csUriQuery5 = replace('userId=[0-9]{5,6}', 'userId=NNN', csUriQuery4)
| extend csUriQuery6 = replace('pageid=[0-9]{1,2}', 'pageid=NNN', csUriQuery5)
| project-away csUriQuery
| project-rename csUriQuery = csUriQuery6
| summarize count() by csUriQuery
| sort by count_ desc
'''
df_iis_csUriQuery = qry_prov.exec_query(query)

In [ ]:
df_iis_csUriQuery.head(20)

In [ ]:
df_iis_csUriQuery.tail(20)

In [ ]:
query = f'''W3CIISLog {query_common_args}
| summarize count() by Computer,sSiteName,sIP,cIP,csMethod,scStatus,csUriStem
| sort by count_ desc
| limit {results_limit}'''
df_iis = qry_prov.exec_query(query)
df_iis.head(results_limit)

In [ ]:
query = f'''W3CIISLog {query_common_args}
| extend is_private = ipv4_is_private(cIP)
| where is_private == True
| summarize count() by cIP
| sort by count_ desc
'''
df_iis_cIP_private = qry_prov.exec_query(query)
df_iis_cIP_private.head(30)

In [ ]:
df_iis_cIP_private.shape

In [ ]:
query = f'''W3CIISLog {query_common_args}
| extend is_private = ipv4_is_private(cIP)
| where is_private == False
| summarize count() by cIP
| sort by count_ desc
'''
df_iis_cIP_public = qry_prov.exec_query(query)
df_iis_cIP_public.head(30)

In [ ]:
df_iis_cIP_public.shape

In [ ]:
query = f'''W3CIISLog {query_common_args}
| where csUriStem has_any ("dump", "tar", "git", ".env") and not (scStatus in (403, 404))
| summarize count() by Computer,sSiteName,sIP,cIP,csMethod,scStatus,csUriStem
| sort by count_ desc
'''
df_q1 = qry_prov.exec_query(query)
df_q1.head(20)

In [ ]:
query = f'''W3CIISLog {query_common_args}
| where (scStatus in (403, 404))
| summarize count() by Computer,sSiteName,sIP,cIP,csMethod,scStatus,csUriStem
| sort by count_ desc
'''
df_q2 = qry_prov.exec_query(query)

In [ ]:
df_q2[(df_q2['scStatus'] == '403')][['cIP', 'csUriStem']].sort_values(
    ['cIP', 'csUriStem'],
    ascending=False
).groupby('csUriStem').count().tail(20)

### Geolocation

In [ ]:
# https://msticpy.readthedocs.io/en/latest/getting_started/Installing.html#selective-installation-using-extras
# https://msticpy.readthedocs.io/en/latest/data_acquisition/GeoIPLookups.html
iplocation = GeoLiteLookup()
# iplocation = IPStackLookup()
df1_geo = iplocation.df_lookup_ip(df_iis_cIP_public, column="cIP")

In [ ]:
df1_geo

In [ ]:
df1_geo.groupby(["City", "CountryCode"]).sum().sort_values(
    by=["CountryCode", "City"], ascending=[True, False]
)

### ASN

In [ ]:
# https://msticpy.readthedocs.io/en/latest/data_acquisition/IPWhois.html#asn-lookups FIXME! docs update
from msticpy.context.ip_utils import get_asn_from_ip

In [ ]:
get_asn_from_ip("65.55.44.109")

In [ ]:
df1_asn['asn'] = df_iis_cIP_public['cIP'].apply(get_asn_from_ip)

In [ ]:
df1_asn

### Web traversal

In [ ]:
query = f'''W3CIISLog {query_common_args}
| where csUriQuery has_any ("..") and
    not (scStatus in ("403")) {vuln_scanner_exclude_and}
| summarize count() by Computer,sSiteName,sIP,cIP,csMethod,scStatus,csUriStem,csUriQuery
| sort by count_ desc
'''
df_web_traversal = qry_prov.exec_query(query)
df_web_traversal.head(20)

### Webshell

In [ ]:
query = f'''DeviceProcessEvents {query_common_args}
| where InitiatingProcessParentFileName in~(
    "beasvc.exe","coldfusion.exe","httpd.exe","owstimer.exe","visualsvnserver.exe","w3wp.exe"
  ) or InitiatingProcessParentFileName startswith "tomcat"
| where InitiatingProcessFileName in~(
    "arp.exe",
    "at.exe",
    "bitsadmin.exe",
    "certutil.exe",
    "cmd.exe",
    "dsget.exe",
    "dsquery.exe",
    "find.exe",
    "findstr.exe",
    "fsutil.exe",
    "hostname.exe",
    "ipconfig.exe",
    "nbstat.exe",
    "net.exe",
    "net1.exe",
    "netdom.exe",
    "netsh.exe",
    "netstat.exe",
    "nltest.exe",
    "nslookup.exe",
    "ntdsutil.exe",
    "pathping.exe",
    "ping.exe",
    "powershell.exe",
    "powershell_ise.exe",
    "qprocess.exe",
    "query.exe",
    "qwinsta.exe",
    "reg.exe",
    "rundll32.exe",
    "sc.exe",
    "schtasks.exe",
    "systeminfo.exe",
    "tasklist.exe",
    "tracert.exe",
    "ver.exe",
    "vssadmin.exe",
    "wevtutil.exe",
    "whoami.exe",
    "wmic.exe",
    "wusa.exe"
  )
| where FileName != 'conhost.exe'
| summarize count() by MachineGroup,DeviceName,InitiatingProcessParentFileName,FolderPath,InitiatingProcessCommandLine
| sort by count_
'''
df_webshell_process = qry_prov.exec_query(query)
df_webshell_process.head(20)

In [ ]:
# 'Cmd=' or 'P=type%20C:\Windows\system.ini%20' on Exchange Web server seems "normal"
query = f'''W3CIISLog {query_common_args}
| where csUriQuery has_any ("dump", "tar", "git", "system", "1=", "2=") and''' + r'''
    not (csUriQuery has_any (":\\Windows\\system.ini"))
| summarize count() by Computer,sSiteName,sIP,cIP,csMethod,scStatus,csUriStem,csUriQuery
| sort by count_ desc
'''
df_webshell_query = qry_prov.exec_query(query)
df_webshell_query.head(20)

In [ ]:
# https://learn.microsoft.com/en-us/troubleshoot/developer/webapps/iis/health-diagnostic-performance/time-taken-field-http-log
# https://techcommunity.microsoft.com/t5/iis-support-blog/time-vs-time-taken-fields-in-iis-logging/ba-p/347677
# https://learn.microsoft.com/en-us/previous-versions/windows/it-pro/windows-server-2003/cc786596(v=ws.10)?redirectedfrom=MSDN
query = f'''W3CIISLog {query_common_args}
| summarize max(TimeTaken),min(TimeTaken),avg(TimeTaken),count() by sSiteName
'''
df_timetaken = qry_prov.exec_query(query)
df_timetaken.head(results_limit)

In [ ]:
query = f'''W3CIISLog {query_common_args}
| summarize max(TimeTaken),min(TimeTaken),avg(TimeTaken),count() by csUriStem
| sort by avg_TimeTaken 
'''
df_timetaken2 = qry_prov.exec_query(query)
df_timetaken2.head(results_limit)

## Visualization

In [ ]:
# Matrix IP, site
query = f'''W3CIISLog {query_common_args}
| summarize count() by sSiteName,cIP
| sort by count_ desc
| limit 100'''
df_ip_site = qry_prov.exec_query(query)

In [ ]:
plot_matrix(
    data=df_ip_site,
    x="sSiteName", y="cIP",
    title="Matrix IP - site"
)

In [ ]:
# Matrix IP, method
query = f'''W3CIISLog {query_common_args}
| summarize count() by cIP,csMethod
| sort by count_ desc
| limit 100'''
df_ip_method = qry_prov.exec_query(query)

In [ ]:
plot_matrix(
    data=df_ip_method,
    x="csMethod", y="cIP",
    title="Matrix IP - method"
)

In [ ]:
# Matrix IP Public, status code
query = f'''W3CIISLog {query_common_args}
| extend is_private = ipv4_is_private(cIP)
| summarize count() by cIP,scStatus,is_private
| sort by count_ desc
'''
df_ip_statuscode = qry_prov.exec_query(query)

In [ ]:
plot_matrix(
    data=df_ip_statuscode[df_ip_statuscode['is_private'] == False].head(100),
    x="scStatus", y="cIP",
    title="Matrix IP public - scStatus"
)

In [ ]:
# Matrix IP Private, status code
plot_matrix(
    data=df_ip_statuscode[df_ip_statuscode['is_private'] == True].head(100),
    x="scStatus", y="cIP",
    title="Matrix IP private - scStatus"
)

In [ ]:
# Matrix IP, status code 403 or 404
plot_matrix(
    data=df_ip_statuscode[(df_ip_statuscode['scStatus'] == '403') | (df_ip_statuscode['scStatus'] == '404')].head(100),
    x="scStatus", y="cIP",
    title="Matrix IP - site"
)

## Misc

In [ ]:
# Noise?

In [ ]:
# Anomalies? no site, computer
query = f'''W3CIISLog {query_common_args}
| where Computer == "" or sSiteName == ""
| summarize count() by Computer,sSiteName,_ResourceId
| sort by count_ desc
'''
df_iis_anomalies = qry_prov.exec_query(query)
df_iis_anomalies.head(30)

In [ ]:
# PII, credentials detection? many more variants...
query = f'''search in (W3CIISLog) ("--password" or "password=" or "_PASSWORD" or "PASSWORD_" or "credentials=" or "pin=" or "cvv=" or "hl7-org") {query_common_args}
| summarize count() by sSiteName,scStatus,csUriStem,csUriQuery
| sort by count_ desc 
| limit {results_limit}'''
df_sensitivedata = qry_prov.exec_query(query)
df_sensitivedata.head(results_limit)